In [20]:
import math                       
import matplotlib.pyplot as plt   
import random                     
import numpy as np                
import numpy.linalg as lin        
import scipy.special as ss
import pickle
from cgi import test
from copy import deepcopy

In [21]:
def get_mag(A):
  return np.sum(A)

def distance(A,B):
  distance = sum(np.logical_and(A,B))
  return distance

def perm(A,N):
  return np.roll(A,N)

In [22]:
def u_gen_rand_hv(D,d):
    # Sanity checker
    if (D % 2):
        print("Error - D can't be an odd number")
        return 0
    # generate
    chosen = random.sample(range(D), k =int(D*d))
    rand_hv = np.array([1 if x in chosen else 0 for x in range(D)])
    return rand_hv

def create_item_mem(N,D,d):
  # insert nice code here
  item_mem = {}
  first = u_gen_rand_hv(D,d)
  item_mem.update({0:first})
  last = u_gen_rand_hv(D,d)
  item_mem.update({N-1:last})
  for i in range(1,N-1):
    temp_hv = np.concatenate((first[:int((D/N)*(N-i))],last[int((D/N)*(N-i)):])) 
    item_mem.update({i:temp_hv})    
  return item_mem

def bundle(block,t): # t - threshold
  if((len(block) % 2) == 0):
    print("Error! Block size should be odd.")
    return
  sum = np.sum(block,axis=0)
  bndl = np.where(sum>t,1,0)
  return bndl

In [23]:
print([1,4]+[4,3,2])

[1, 4, 4, 3, 2]


In [24]:
h = u_gen_rand_hv(20,0.5)
print(h)
chs = random.sample(range(20), k =int(5))
hi = [not h[x] if x in chs else h[x] for x in range(20)]
print(hi)
print(np.array(hi))

[0 0 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 1]
[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, True, True, False, 1, True, True, 1, 1, 1]
[0 0 0 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1]


In [25]:
def hdc_encode(voice,voice_im,D,d,Q=10, remove_idx = []): #d = density, Q - quntization steps
    voice = [ int(math.floor(x*(Q/2))+(Q/2)) if x<1 else int(math.floor((x-0.0001)*(Q/2))+(Q/2)) for x in voice]
    feature_hv_list = [voice_im[x] for x in voice]
    feature_hv_list = np.array([perm(feature_hv_list[x],x) for x in range(len(feature_hv_list))])
    
    threshold = 36 #first threshold
    bundle = np.sum(feature_hv_list,axis=0)
    
    if remove_idx:
        bundle = np.delete(bundle, remove_idx)
        
    #for threshold in range(1000):
    bndl = np.where(bundle>threshold,1,0)
    #    print(threshold,sum(bndl))
    return bndl

In [26]:
def redundancy_pruning(bin_reg, remove_idx):
  comp = enumerate(np.where(bin_reg[0]==bin_reg[1], bin_reg[0], "False"))
  remove = [i for i in comp if i[1]!="False"]
  new_list = []
  
  for i in bin_reg[2::]:
    for j in remove:
      if int(j[1]) == int(i[int(j[0])]):
        new_list.append(j)
    remove = new_list
    new_list = []
  
  for i in remove:
    remove_idx.append(i[0])
    
  #delete the specified indices in all of the classes
  for i in range(len(bin_reg)):
    bin_reg[i] = np.delete(bin_reg[i], remove_idx)

In [27]:
D = 5000
Q = 10
d = 0.05
remove_idx = []

voice_im = create_item_mem(Q,D,d)
for i in range(10):
    print(i,distance(voice_im[0],voice_im[i]))

with open('isolet.pkl', 'rb') as f:
    isolet = pickle.load(f)
trainData, trainLabels, testData, testLabels = isolet
trainData = np.array(trainData)
testData = np.array(testData)

voice_am = dict()

0 250
1 225
2 205
3 178
4 149
5 124
6 93
7 79
8 60
9 15


In [28]:
for i in range(1):
    print(sum(hdc_encode(trainData[65],voice_im,D,d,Q)))

807


In [29]:
non_bin_reg = np.zeros((26,D))
#print(hdc_encode(trainData[i],voice_im, D, d, Q))
for i in range(len(trainLabels)):
    non_bin_reg[trainLabels[i]] = np.add(non_bin_reg[trainLabels[i]],hdc_encode(trainData[i],voice_im, D, d, Q))
    
print(non_bin_reg)

[[136. 154. 135. ... 190. 127. 103.]
 [ 85. 127. 133. ... 169.  97.  86.]
 [ 59.  55.   9. ... 166.  90.  53.]
 ...
 [128. 209. 235. ... 214. 225. 203.]
 [ 37.  41.  89. ... 106.  65.  72.]
 [ 46.  66.  16. ... 140. 150.  73.]]


In [31]:
threshold2 = 40 #second threshold
for i in range(26):
    voice_am[i] = np.where(non_bin_reg[i]>threshold2,1,0)

bin_reg = []
for i in voice_am.values():
    bin_reg.append(i)
dimension_sparsifier(bin_reg, remove_idx)

for i in range(26):
    voice_am[i] = bin_reg[i]

In [32]:
def similarity_search(voice,voice_im,voice_am,D,d,Q):
  # insert nice code here
  sim_score = 0
  sim_letter = '0'
  test_hv = hdc_encode(voice,voice_im,D,d,Q, remove_idx)
  for each in voice_am.items():
    #print(each)
    if (distance(test_hv,each[1])>sim_score):
      sim_score,sim_letter = distance(test_hv,each[1]), each[0] 
  return sim_letter, sim_score

In [33]:
def test_model(test_data,correct_values,voice_im,voice_am,D,d,Q,print_flag):

  # Simply iterate through all elements in the clean_letters set
  score = 0
  test_len = len(test_data)
  for i in range(test_len):
    
    sim_letter, sim_score = similarity_search(test_data[i],voice_im,voice_am,D,d,Q)

    
    if sim_letter == correct_values[i]:
      score += 1
      if(print_flag):
        print("CORRECT prediction! sim_letter: ", sim_letter, " sim_score: ", str(sim_score))
    else:
      if(print_flag):
        print("WRONG prediction! sim_letter: " , sim_letter, " sim_score: ", str(sim_score))

  print("Final accuracy is: %f" %((score/test_len)*100))
  return (score/test_len)*100

In [34]:
test_model(testData,testLabels,voice_im,voice_am,D,d,Q,True)

CORRECT prediction! sim_letter:  0  sim_score:  253
WRONG prediction! sim_letter:  8  sim_score:  260
WRONG prediction! sim_letter:  4  sim_score:  301
WRONG prediction! sim_letter:  4  sim_score:  255
CORRECT prediction! sim_letter:  2  sim_score:  258
CORRECT prediction! sim_letter:  2  sim_score:  267
WRONG prediction! sim_letter:  19  sim_score:  277
CORRECT prediction! sim_letter:  3  sim_score:  283
CORRECT prediction! sim_letter:  4  sim_score:  271
CORRECT prediction! sim_letter:  4  sim_score:  269
WRONG prediction! sim_letter:  22  sim_score:  271
WRONG prediction! sim_letter:  22  sim_score:  256
CORRECT prediction! sim_letter:  6  sim_score:  295
WRONG prediction! sim_letter:  15  sim_score:  299
CORRECT prediction! sim_letter:  7  sim_score:  280
CORRECT prediction! sim_letter:  7  sim_score:  295
CORRECT prediction! sim_letter:  8  sim_score:  285
CORRECT prediction! sim_letter:  8  sim_score:  261
CORRECT prediction! sim_letter:  9  sim_score:  273
WRONG prediction! sim_

75.56125721616421